In [1]:
import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import numpy as np
import pickle

data_files = ['dev', 'test', 'train']

data_path = "./data/%s_dataset.json"
x_file = './data/agg_data/%s_agg_X.pkl'
y_file = './data/agg_data/%s_agg_Y.pkl'
idx_file = './data/op_data/%s_op_idx.pkl'

def get_data(dt):
    with open(y_file%dt, 'rb') as fy:
        _Y = pickle.load(fy)

    with open(x_file%dt, 'rb') as fx:
        _X = pickle.load(fx)
    return (_X, _Y)

dev_X, dev_Y = get_data('train')


def get_batch(idx, batch_size):
    lw = idx * batch_size
    hs = lw + batch_size
    if hs < len(dev_X):
        return (dev_X[lw:hs][:], dev_Y[lw:hs][:])
    else:
        return (dev_X[lw:][:], dev_Y[lw:][:])

# def train():
n_input = 1500
n_hidden1 = 1024
n_hidden2 = 512
n_hidden3 = 64
n_output = 6

learning_rate = 1e-4
n_iteration = 101
batch_size = 128
dropout = 0.1

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
# keep_prob = tf.placeholder(tf.float32) # used to control the dropout rate

weights = {
'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1))
}

biases = {
'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

layer_1 = tf.nn.tanh(tf.add(tf.matmul(X, weights['w1']), biases['b1']))
layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['w3']), biases['b3']))
# layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.nn.leaky_relu(tf.add(tf.matmul(layer_3, weights['out']), biases['out']))

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)



n_batches = len(dev_X)//batch_size

# train on mini batches
for i in range(n_iteration):
    for j in range(n_batches+1):
        (batch_x, batch_y) = get_batch(j, batch_size)
        sess.run(train_step, feed_dict={X: batch_x, Y: batch_y})

        # print loss and accuracy (per minibatch)
    if i % 5 == 0:
        minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y})
        print("Iteration", str(i), "\t| Batch", str(j), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))

tf.train.Saver().save(sess, './checkpoints/agg/training.ckpt')

test_X, test_Y = get_data('test')
test_accuracy = sess.run(accuracy, feed_dict={X: test_X, Y: test_Y})
print("\nAccuracy on test set : ", test_accuracy)

Iteration 0 	| Batch 127 	| Loss = 0.72512895 	| Accuracy = 0.75757575
Iteration 5 	| Batch 127 	| Loss = 0.50767314 	| Accuracy = 0.82828283
Iteration 10 	| Batch 127 	| Loss = 0.42340434 	| Accuracy = 0.8888889
Iteration 15 	| Batch 127 	| Loss = 0.37670678 	| Accuracy = 0.8888889
Iteration 20 	| Batch 127 	| Loss = 0.35165134 	| Accuracy = 0.8989899
Iteration 25 	| Batch 127 	| Loss = 0.3336843 	| Accuracy = 0.8888889
Iteration 30 	| Batch 127 	| Loss = 0.32248512 	| Accuracy = 0.8888889
Iteration 35 	| Batch 127 	| Loss = 0.3161664 	| Accuracy = 0.8888889
Iteration 40 	| Batch 127 	| Loss = 0.3096173 	| Accuracy = 0.8888889
Iteration 45 	| Batch 127 	| Loss = 0.30564573 	| Accuracy = 0.8888889
Iteration 50 	| Batch 127 	| Loss = 0.30344027 	| Accuracy = 0.8989899
Iteration 55 	| Batch 127 	| Loss = 0.30222684 	| Accuracy = 0.8888889
Iteration 60 	| Batch 127 	| Loss = 0.30030176 	| Accuracy = 0.8888889
Iteration 65 	| Batch 127 	| Loss = 0.2985212 	| Accuracy = 0.8888889
Iteration 

In [ ]:
agg_ops = ['', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']